# 0.0. Imports

In [1]:
import pickle
import inflection

import numpy as np
import pandas as pd
import seaborn as sn

from xgboost import XGBClassifier
from ydata_profiling import ProfileReport

from sklearn import metrics as mt
from sklearn import ensemble as en
from sklearn import model_selection as ms
from sklearn import preprocessing   as pp

from sklearn import linear_model as lm
from sklearn import ensemble as en


/home/felipe/repos/hackday/hackday/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1.0. Data Description

## 1.1. Loading Data

In [2]:
df_raw = pd.read_csv('data/test.csv')
df_country = pd.read_csv('data/country_base_2.csv')

In [3]:
df1 = pd.merge(df_raw, df_country, on='Nacionalidade', how='left')

In [4]:
df1 = df1.drop('country', axis=1)

## 1.2. Rename Columns

In [5]:
cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
            'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
            'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
            'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes', 'latitude', 'longitude']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))
df1.columns = cols_new

## 1.3. Data Dimensions

In [6]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of cols: {}'.format(df1.shape[1]))

Number of rows: 48106
Number of cols: 16


## 1.4. Check NA

In [7]:
df1.isna().sum()#/len(df1)

id                                        0
classificacao_do_hotel                    0
meses_da_reserva_ate_o_check_in           0
numero_de_pernoites_reservadas            0
numero_de_hospedes                        1
regime_de_alimentacao                     0
nacionalidade                           677
forma_de_reserva                          0
ja_se_hospedou_anterioremente             0
tipo_do_quarto_reservado                  0
reserva_feita_por_agencia_de_turismo      0
reserva_feita_por_empresa                 0
reserva_com_estacionamento                0
reserva_com_observacoes                   0
latitude                                677
longitude                               677
dtype: int64

### 1.4.1. Replace NA

In [8]:
df1['numero_de_hospedes'] = df1['numero_de_hospedes'].fillna(2)
df1['nacionalidade'] = df1['nacionalidade'].fillna('Spain')
df1['latitude'] = df1['latitude'].fillna(40463667)
df1['longitude'] = df1['longitude'].fillna(-374922)

In [9]:
# import pandas as pd
# from sklearn.linear_model import LinearRegression
# from sklearn.impute import SimpleImputer

# # carregar dados de exemplo com valores ausentes
# df = pd.read_csv('exemplo.csv')

# # separar os dados em variáveis independentes e dependentes
# X = df.drop('Target', axis=1)
# y = df['Target']

# # criar um objeto SimpleImputer para imputar valores ausentes
# imputer = SimpleImputer(strategy='mean')

# # imputar os valores ausentes usando regressão linear
# for column in X.columns:
#     X_column = X[[column]]
#     X_column_with_nulls = X_column[X_column[column].isnull()]
#     X_column_without_nulls = X_column[X_column[column].notnull()]
#     y_without_nulls = y[X_column[column].notnull()]
#     model = LinearRegression()
#     model.fit(X_column_without_nulls, y_without_nulls)
#     y_pred = model.predict(X_column_with_nulls)
#     X_column_with_nulls[column] = y_pred
#     X[[column]] = pd.concat([X_column_with_nulls, X_column_without_nulls], axis=0)

# # juntar as variáveis independentes e dependentes de volta em um dataframe
# df_imputed = pd.concat([X, y], axis=1)

# # exibir o dataframe resultante
# print(df_imputed)

## 1.5. Data Dtypes

In [10]:
df1.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
latitude                                float64
longitude                               float64
dtype: object

### 1.5.1. Change Dtypes

In [11]:
df1['numero_de_hospedes'] = df1['numero_de_hospedes'].astype('int')

## 1.6. Data Balancing

In [12]:
#df1['reserva_cancelada'].value_counts(normalize=True)

## 1.7. Pandas Profiling

In [13]:
# prof = ProfileReport(df_raw)
# prof.to_file(output_file='output.html')

# 2.0. Feature Engineering

In [14]:
df2 = df1.copy()

In [15]:
df2['classificacao_do_hotel'] = df2['classificacao_do_hotel'].apply(lambda x: x.replace(' estrelas', ''))

In [16]:
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Sim', '1'))
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Não', '0'))

In [17]:
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Não', '0'))

In [18]:
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].apply(lambda x: x.replace('Não', '0'))

In [19]:
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].apply(lambda x: x.replace('Não', '0'))

In [20]:
df2['classificacao_do_hotel'] = df2['classificacao_do_hotel'].astype('int')
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].astype('int')
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].astype('int')
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].astype('int')
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].astype('int')

# 3.0. Data Filtering

In [21]:
df3 = df2.copy()

In [22]:
# reserva_feita_por_agencia_de_turismo reserva_feita_por_empresa
# df3 = df3.drop(['reserva_feita_por_agencia_de_turismo', 'reserva_feita_por_empresa'], axis=1)

# 4.0. Data Preparation 

In [23]:
#df4 = df3.drop('nacionalidade', axis=1).copy()
df4 = df3.copy()

In [24]:
#tipo_do_quarto_reservado
# criar um dicionário de frequência de cada categoria
freq_dict = df4['tipo_do_quarto_reservado'].value_counts(normalize=True).to_dict()
# aplicar o encoder ao dataframe
df4['tipo_do_quarto_reservado_encoded'] = df4['tipo_do_quarto_reservado'].map(freq_dict)

In [25]:
df4.head()

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,latitude,longitude,tipo_do_quarto_reservado_encoded
0,118345,4,8,13,3,Café da manha,South Korea,Balcão,0,Green Emerald,1,0,0,Nenhuma,35907757.0,127766922.0,0.157964
1,9500,5,5,4,769,Café da manha,United States,Agência,0,Pink Sapphire,1,0,0,Nenhuma,3709024.0,-95712891.0,0.054920
2,34558,5,4,11,1,Café da manha e jantar,United Kingdom,Agência,0,Amethyst,0,0,0,Nenhuma,55378051.0,-3435973.0,0.723008
3,70816,4,7,4,2,Café da manha,Spain,Agência,0,Amethyst,0,0,0,Nenhuma,40463667.0,-374922.0,0.723008
4,105321,4,1,8,2,Café da manha,Italy,B2B,0,Amethyst,0,1,0,1 a 3,4187194.0,1256738.0,0.723008


In [26]:
# dummy variables
cols_dummy = ['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes']

df4_dummy = pd.get_dummies(df4[cols_dummy])

df4 = pd.concat([df4, df4_dummy], axis=1)

# # numerical
# col_num = df4.select_dtypes(include=['int64', 'float64']).columns.tolist()

# # categorical
# col_cat = df4.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]']).columns.tolist()

# # encoding
# df4_dummy = pd.DataFrame(ohe.fit_transform(df4[col_cat]).toarray(), index=df4.index)
# pickle.dump(ohe, open('ohe.pkl', 'wb' ))

# # join numerical and categorical
# df42 = pd.concat([df4[col_num], df4_dummy], axis=1)
# df42.columns = df42.columns.astype(str)

# # one hot encoding
# ohe_class_hotel = pp.OneHotEncoder()
# ohe_aliment = pp.OneHotEncoder()
# ohe_form_reserv = pp.OneHotEncoder()
# ohe_hosp_anter = pp.OneHotEncoder()
# ohe_reserv_estac = pp.OneHotEncoder()
# ohe_reserva_observ = pp.OneHotEncoder()

# # classificacao_do_hotel
# ohe_class_hotel.fit(df4[['classificacao_do_hotel']])
# pickle.dump(ohe_class_hotel, open('ohe_class_hotel.pkl', 'wb' ))
# encoded = ohe_class_hotel.transform(df4[['classificacao_do_hotel']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_class_hotel.get_feature_names_out(['classificacao_do_hotel']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # regime_de_alimentacao
# ohe_aliment.fit(df4[['regime_de_alimentacao']])
# pickle.dump(ohe_aliment, open('ohe_aliment.pkl', 'wb' ))
# encoded = ohe_aliment.transform(df4[['regime_de_alimentacao']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_aliment.get_feature_names_out(['regime_de_alimentacao']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # forma_de_reserva
# ohe_form_reserv.fit(df4[['forma_de_reserva']])
# pickle.dump(ohe_form_reserv, open('ohe_form_reserv.pkl', 'wb' ))
# encoded = ohe_form_reserv.transform(df4[['forma_de_reserva']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_form_reserv.get_feature_names_out(['forma_de_reserva']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # ja_se_hospedou_anterioremente
# ohe_hosp_anter.fit(df4[['ja_se_hospedou_anterioremente']])
# pickle.dump(ohe_hosp_anter, open('ohe_hosp_anter.pkl', 'wb' ))
# encoded = ohe_hosp_anter.transform(df4[['ja_se_hospedou_anterioremente']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_hosp_anter.get_feature_names_out(['ja_se_hospedou_anterioremente']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # reserva_com_estacionamento
# ohe_reserv_estac.fit(df4[['reserva_com_estacionamento']])
# pickle.dump(ohe_reserv_estac, open('ohe_reserv_estac.pkl', 'wb' ))
# encoded = ohe_reserv_estac.transform(df4[['reserva_com_estacionamento']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_reserv_estac.get_feature_names_out(['reserva_com_estacionamento']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# #reserva_com_observacoes
# ohe_reserva_observ.fit(df4[['reserva_com_observacoes']])
# pickle.dump(ohe_reserva_observ, open('ohe_reserva_observ.pkl', 'wb' ))
# encoded = ohe_reserva_observ.transform(df4[['reserva_com_observacoes']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_reserva_observ.get_feature_names_out(['reserva_com_observacoes']))
# df4 = pd.concat([df4, df_encoded], axis=1)

In [27]:
# df4 = df4.drop(['tipo_do_quarto_reservado', 'classificacao_do_hotel', 'regime_de_alimentacao', 'forma_de_reserva',
#          'ja_se_hospedou_anterioremente', 'reserva_com_estacionamento', 'reserva_com_observacoes'], axis=1)


In [28]:
df4['latitude'] = df4['latitude'].apply(lambda x: np.radians(x/1000000))
df4['longitude'] = df4['longitude'].apply(lambda x: np.radians(x/1000000))

# 5.0. Feature Selection 

In [29]:
df5 = df4.copy()
cols_drop =['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes', 'nacionalidade', 'tipo_do_quarto_reservado']
df5 = df4.drop(cols_drop, axis=1)

In [30]:
# split dataset into features and target
# df5 = df5.drop('id', axis=1)

In [31]:
# # split dataset into train and test
# X_train, X_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.2)

## 1.5. Data Dtypes

# 6.0. Machine Learning Model

## 6.1. XGBoost

In [32]:
xgb_model = pickle.load(open('xgb_model.pkl', 'rb'))
#model = pickle.load(open('rf_model.pkl', 'rb'))

In [33]:
yhat_xbg = xgb_model.predict(df5)

## 6.1. Gradient Boost

In [34]:
gdr_model = pickle.load(open('gdr_model.pkl', 'rb'))
#model = pickle.load(open('rf_model.pkl', 'rb'))

In [35]:
yhat_gb = gdr_model.predict(df5)

## 6.1. Random Forest

In [36]:
rf_model = pickle.load(open('rf_model.pkl', 'rb'))
#model = pickle.load(open('rf_model.pkl', 'rb'))

In [37]:
yhat_rf = rf_model.predict(df5)

## 6.1. Ensemble

In [39]:
ensemble = pd.DataFrame([yhat_xbg, yhat_gb, yhat_rf]).T
ensemble.columns = ['yhat_xgb', 'yhat_gb', 'yhat_rf']
moda = ensemble.mode(axis=1)[0]
ensemble['pred'] = moda

In [40]:
yhat_en = ensemble['pred']

# 7.0. Export CSV

In [41]:
df2['Reserva Cancelada'] = yhat_en

In [42]:
submission = df2[['id', 'Reserva Cancelada']].copy()

In [43]:
submission.to_csv('submission.csv', index=False)